In [1]:
import time
import weaviate

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.select import Select
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver import ActionChains

In [2]:
options = Options()
options.add_argument("--headless")
options.add_argument("start-maximized")
options.add_argument('--disable-dev-shm-usage')
# 리눅스 서버같은 GUI환경이 지원안될 때 필요한 옵션
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

url = 'https://whiskyadvocate.com/ratings-reviews'
driver = webdriver.Chrome(options=options)
# options.add_argument('--remote-debugging-port=8903')

In [3]:
minscore = 90
maxscore = 99
minprice = 1
maxprice = 999

eleClass = 'mwInput.checkList.new.filter.select.multiline.noMobile.name-filtersdefaultcustom_rating_category.list'
subBtnEle = 'submit.button.mwFilter-submit.mwFormSubmit'
issueClass = 'mwInput.checkList.new.filter.select.multiline.noMobile.name-filtersdefaultcustom_rating_issue.list'

def func_filtering() :
    # 필터링 (가격, 점수)
    minScore = driver.find_element(By.NAME, 'filters[default][min_score]')
    driver.execute_script(f"arguments[0].setAttribute('value', '{minscore}');", minScore)
    maxScore = driver.find_element(By.NAME, 'filters[default][max_score]')
    driver.execute_script(f"arguments[0].setAttribute('value', '{maxscore}');", maxScore)

    minPrice = driver.find_element(By.NAME, 'filters[default][min_price]')
    driver.execute_script(f"arguments[0].setAttribute('value', '{minprice}')", minPrice)
    maxPrice = driver.find_element(By.NAME, 'filters[default][max_price]')
    driver.execute_script(f"arguments[0].setAttribute('value', '{maxprice}')", maxPrice)

# 필요한 Elements 가져오기
def func_findEle(eleClass) :
    getElements = driver.find_element(By.CLASS_NAME, eleClass)

    # 리스트 가져오기
    getList = getElements.find_element(By.CLASS_NAME, 'list')
    child_divs = getList.find_elements(By.TAG_NAME, 'div')

    # select box option 가져오기
    select_element = driver.find_element(By.NAME, 'filters[default][custom_rating_category][]')
    select = Select(select_element)
    option_list = select.options

    func_filtering()

    return child_divs, option_list

def func_findIssueEle(issueClass) :
    getElements = driver.find_element(By.CLASS_NAME, issueClass)

    # 리스트 가져오기
    getList = getElements.find_element(By.CLASS_NAME, 'list')
    child_issueDivs = getList.find_elements(By.TAG_NAME, 'div')

    # select box option 가져오기
    select_element = driver.find_element(By.NAME, 'filters[default][custom_rating_issue][]')
    select = Select(select_element)
    issueOption_list = select.options

    func_filtering()

    return child_issueDivs, issueOption_list

def func_saveUrl() :
    urlList = []
    time.sleep(5)

    currentUrl = driver.current_url
    urlList.append(currentUrl)

    time.sleep(5)

    return urlList


# 위스키 스타일을 선택. 선택한 option별로 url이 달라지기 때문에 옵션 선택 별로 url 저장
def func_selectStyle(url, eleClass, issueClass, subBtnEle) :
    driver.get(url)

    child_divs, option_list = func_findEle(eleClass)

    for i in range(1, len(child_divs)) :
        list = child_divs[i]
        opt = option_list[i]
        action = ActionChains(driver)
        action.move_to_element(driver.find_element(By.CLASS_NAME, eleClass)).perform()

        driver.execute_script("arguments[0].classList.add('selected');", list)
        driver.execute_script("arguments[0].setAttribute('selected', 'selected');", opt)

        action.move_to_element(driver.find_element(By.CLASS_NAME, subBtnEle)).perform()
        searchBtn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, subBtnEle)))

        searchBtn.click()

        time.sleep(5)
        resCnt = int(driver.find_element(By.CLASS_NAME, 'resultsNumber').text.split()[0])
        
        # 결과가 200개 밖에 나오지 않아서 처리해주기 위한 곳
        if resCnt > 200 :
            child_issueDivs, issueOption_list = func_findIssueEle(issueClass)

            for j in range(1, len(child_issueDivs)) :
                list = child_issueDivs[j]
                opt = issueOption_list[j]
                action = ActionChains(driver)
                action.move_to_element(driver.find_element(By.CLASS_NAME, issueClass)).perform()

                driver.execute_script("arguments[0].classList.add('selected');", list)
                driver.execute_script("arguments[0].setAttribute('selected', 'selected');", opt)

                action.move_to_element(driver.find_element(By.CLASS_NAME, subBtnEle)).perform()
                searchBtn = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CLASS_NAME, subBtnEle)))

                searchBtn.click()

                urlList = func_saveUrl()

                action.move_to_element(driver.find_element(By.CLASS_NAME, issueClass)).perform()

                resetBtn = driver.find_element(By.CLASS_NAME, 'reset.button.mwFilter-reset.mwFormSubmit')
                resetBtn.click()
                time.sleep(5)
                
                child_issueDivs, issueOption_list = func_findIssueEle(issueClass)

        else : 
            urlList = func_saveUrl()
            
            action.move_to_element(driver.find_element(By.CLASS_NAME, eleClass)).perform()

            resetBtn = driver.find_element(By.CLASS_NAME, 'reset.button.mwFilter-reset.mwFormSubmit')
            resetBtn.click()
            time.sleep(5)
            
            child_divs, option_list = func_findEle(eleClass)

    return urlList

optionUrlList = func_selectStyle(url, eleClass, issueClass, subBtnEle)

WebDriverException: Message: unknown error: session deleted because of page crash
from unknown error: cannot determine loading status
from tab crashed
  (Session info: chrome=129.0.6668.58)
Stacktrace:
#0 0x55b330c2602a <unknown>
#1 0x55b33090c43d <unknown>
#2 0x55b3308f5bc1 <unknown>
#3 0x55b3308f4d9e <unknown>
#4 0x55b3308f452f <unknown>
#5 0x55b3308f4422 <unknown>
#6 0x55b3308f2493 <unknown>
#7 0x55b3308f2c4f <unknown>
#8 0x55b330902667 <unknown>
#9 0x55b3309189b4 <unknown>
#10 0x55b33091ddfb <unknown>
#11 0x55b3308f335e <unknown>
#12 0x55b330918501 <unknown>
#13 0x55b33099fc46 <unknown>
#14 0x55b3309808c3 <unknown>
#15 0x55b33094e6b3 <unknown>
#16 0x55b33094f68e <unknown>
#17 0x55b330bf0a2b <unknown>
#18 0x55b330bf49b1 <unknown>
#19 0x55b330bdd225 <unknown>
#20 0x55b330bf5532 <unknown>
#21 0x55b330bc238f <unknown>
#22 0x55b330c14f28 <unknown>
#23 0x55b330c150f3 <unknown>
#24 0x55b330c24e7c <unknown>
#25 0x7f2b9cfaaa94 <unknown>
#26 0x7f2b9d037c3c <unknown>


In [4]:
print(optionUrlList)

['https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=American+Malt+Whiskey', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=American+Whiskey', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Australian+Whisky', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Austrian+Whisky', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Belgian+Whisky', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Bhutanese+Whisky', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Blended+Scotch+Whisky', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Blended+American+Whiskey', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Blended+Belgian+Whisky', 'https://whiskyadvocate.com/ratings-reviews?custom_rating_category%5B0%5D=Blended+French+Whisky', 'https://whiskyadvocate.com/ratings-review